# MBTI별 포스팅 모으기

In [17]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [18]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

#input_file = r'.\data\training'
#allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
#for file in allFile_list:
#    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
#    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

# tokenize and stemming

In [41]:
mbti= dataCombine
X_data = mbti['posts']
y_data = mbti['type']

print(X_data.shape)
print(y_data.shape)

print("normalize 시작")
normalized_text = []
for sentence in X_data.tolist():
    clean_sentence = re.sub('[^A-Za-z\s]+', '',sentence.lower())    # posts 데이터 정규화()
    normalized_text.append(clean_sentence)
mbti.posts = normalized_text
print("normalize 완료")

(8675,)
(8675,)
normalize 시작
normalize 완료


In [42]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

stemData=[] # stemmed X_data: mbti['posts']
tokenizedData=[]    # tokenized X_data
for sentence in mbti['posts']:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []   # 개별 post
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    tokenizedData.append(tokenData)
    stemData.append(tempData)
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]


# vectorization (tfidf)

In [44]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())

[[0.15619188 0.         0.         ... 0.         0.         0.        ]
 [0.22128246 0.04019078 0.         ... 0.         0.         0.        ]
 [0.14145265 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.18796687 0.         0.         ... 0.         0.         0.        ]
 [0.19099817 0.         0.         ... 0.         0.         0.        ]
 [0.13217717 0.         0.         ... 0.         0.         0.        ]]


# Garbage Collection

In [45]:
import psutil
import os
import gc

In [46]:
'''del [[dataCombine]]'''

'''del [condition]'''

'''for ownsentence in allData:
    del ownsentence
del allData'''

'''for tokenData in tokenizedData:   # delete 'stemData'
    for data in tokenData:
        del data
    del tokenData
del tokenizedData'''

'''for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text'''

'''del X_data'''

'''for component in y_data:
    del component
del y_data'''

'''for data in stemData:   # delete 'stemData'
    for component in data:
        del component
    del data
del stemData'''

gc.collect()

49386

In [52]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)

In [54]:
from tensorflow.keras import regularizers

model = Sequential()
model.add(LSTM(128, return_sequences=True))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(64, 'relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(128, 'relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split = 0.2)

Epoch 1/5


ValueError: in user code:

    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    C:\Python38\lib\site-packages\tens

In [ ]:
model.save('.\\models\\LSTM_rawsample.h5')

In [ ]:
model.summary()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=32)